# Starbucks Capstone Challenge - Predictive and Modelling

## Introduction

Offer Selection

* Remove unsuccessful offers
* Strip the ID and perform a classifier

Receives Offer or Not

* Receives offer if, offer successful and total spent is 20 (10 from promo, + additional product) 

Combination of both will give better results in incremental profit and IRR.

## Setup

In [1]:
import sys

!{sys.executable} -m pip install -e ../ --quiet

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sb_capstone.wrangling import (
    clean_transcript_group
)

%matplotlib inline

In [3]:
transcript_group = pd.read_csv("../data/processed/transcript_group.csv")
transcript_group = clean_transcript_group(transcript_group)

transcript_group.head()

,id,wave,diffs,received,viewed,completed,amount,reward,non_offer_amount,mapped_offer,...,mobile,social,gender,age,income,membership_year,membership_month,membership_day,generation,group
0,1,2,48.000000,True,True,False,0.00,0.0,0.00,10,...,True,False,U,NaN,NaN,2017,2,12,NaN,NaN
1,2,2,49.866737,False,False,False,0.00,0.0,0.00,0,...,False,False,F,55.0,112000.0,2017,7,15,boomers,middle_age
2,3,2,36.000000,True,True,False,0.00,0.0,0.00,4,...,True,False,U,NaN,NaN,2018,7,12,NaN,NaN
3,4,2,26.000000,True,True,True,19.67,0.0,29.72,8,...,True,True,F,75.0,100000.0,2017,5,9,silent,old
4,5,2,49.866737,False,False,False,0.00,0.0,3.50,0,...,False,False,U,NaN,NaN,2017,8,4,NaN,NaN


In [4]:
transcript_group.dtypes

id                     int64
wave                   int64
diffs                float64
received                bool
viewed                  bool
completed               bool
amount               float64
reward               float64
non_offer_amount     float64
mapped_offer        category
offer_type          category
difficulty           float64
duration             float64
web                     bool
email                   bool
mobile                  bool
social                  bool
gender              category
age                  float64
income               float64
membership_year        int64
membership_month       int64
membership_day         int64
generation            object
group                 object
dtype: object

## Q3. How can we make use of the result to improve the app using machine learning?